In [1]:
import sksurv as sks
import sksurv.preprocessing
import sksurv.metrics
import sksurv.datasets
import sksurv.linear_model
import sksurv.ensemble

from pathlib import Path
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy as np
import sklearn as skl
import scipy.stats as sp

import pymc as pm
import pymc_bart as pmb
import pandas as pd

import importlib
import mlflow as ml
import simsurv_func as ssf

In [2]:
plt.ioff()
np.random.seed(99)

In [14]:

importlib.reload(ssf)

<module 'simsurv_func' from '/home/twj8/pymc_bart/simsurv_func.py'>

# Objective
Simulations of the cox proportional and non proportional survial models. 
Validation of bart pymc against cph model, rsf model.

Validation metrics:
    2 bool
    5 bool
    2 bool + 1 linear
    complex combination



In [ ]:
# Set experiment
    # - each simulation parms is a new experiment

# Simulation loop
    # Creat run
        # Simulate data
        # log param alpha, lambda
        # log param N
        # log param T (of probabilites generated)
        # log param x_info
        # log param cen percent calculated
        # log param status event calculated
        # log param t_event mean
        # log param t_event max
        # log artif train dataset
        # log artif plot curves

        # model cph
        # log metri coeff
        # log metri exp(coef)
        # log artif plot curves
        # log model cph
        
        #  model rsf
        # log artif plot curves
        # log model resf

        # tranform data long-form
        # model bart
        # transform to survival
        # log artif plot curves
        # log model bart

        # get metrics rmse, bias
        # log metri cph_rmse
        # log metri cph_bias
        # log metri rsf_rmse
        # log metri rsf_bias
        # log metri bart_rmse
        # log metri bart_bias
    
    # End run





In [26]:
ml.autolog(False)

2023/08/01 16:21:35 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


In [12]:
# ml.set_experiment("test_sim")
ml.active_run()

In [28]:
# Set experiment
    # - each simulation parms is a new experiment

try:
    ml.create_experiment(name = "test_sim")
except:
    ml.set_experiment("test_sim")

# Simulation loop

    # Creat run
with ml.start_run(run_name="test1") as run:
    run_info = ml.active_run()
    OUTPUTS = "outputs"
    ALPHA = 3
    LAMBDA = "np.exp(2 + 0.3*(x_mat[:,0] + x_mat[:,1]))"
    N = 1000
    T = 30
    X_VARS = 2
    CENS_SCALE = 60
    # Simulate data
    sv_mat, x_mat, lmbda, a, tlat, cens, t_event, status = ssf.sim_surv(
                    N=N, 
                    T=T,
                    x_vars=X_VARS,
                    a = ALPHA,
                    lambda_f = LAMBDA,
                    cens_scale=CENS_SCALE,
                    cens_ind = True,
                    err_ind = False)

    # log param alpha
    ml.log_param("alpha", ALPHA)
    # log param labmda
    ml.log_param("lambda", LAMBDA)
    # log param N
    ml.log_param("N", N)
    # log param T (# timepoint probabilites generated)
    ml.log_param("T", T)
    # log param X_VARS
    ml.log_param("X_VARS", X_VARS)
    # log parm CENS_SCALE
    ml.log_param("CENS_SCALE", CENS_SCALE)
    
    
    # log param x_info
    x_out, x_idx, x_cnt = ssf.get_x_info(x_mat)
    ml.log_param("X_INFO", str(list(zip(x_out, x_cnt))))

    # log metric cen percent calculated
    # log metric status event calculated
    event_calc, cens_calc = ssf.get_status_perc(status)
    ml.log_metric("EVENT_PERC", event_calc)
    ml.log_metric("CENS_PERC", cens_calc)

    # log metric t_event mean
    # log metric t_event max
    t_mean, t_max = ssf.get_event_time_metric(t_event)
    ml.log_metric("T_EVENT_MEAN", t_mean)
    ml.log_metric("T_EVENT_MAX", t_max)
    
    # log artif train dataset
    train = ssf.get_train_matrix(x_mat, t_event, status)
    train.to_csv("outputs/train.csv")
    ml.log_artifact("outputs/train.csv")
    
    # log artif plot curves
    title = "actual_survival"
    ssf.plot_sv(x_mat, sv_mat, T, title=title, save = True, dir=OUTPUTS)
    ml.log_artifact(f"{OUTPUTS}/{title}.png")
    

    # get sklearn components
    y_sk = ssf.get_y_sklearn(status, t_event)
    x_sk = train.iloc[:,2:]

    ######################################################################################
    # model cph
    cph = sksurv.linear_model.CoxPHSurvivalAnalysis()
    cph.fit(x_sk, y_sk)
    # log metri coeff
    for i in np.arange(len(cph.coef_)):
        ml.log_metric(f"cph_coef_{i}", cph.coef_[i])
        # log metri exp(coef)
        ml.log_metric(f"cph_exp_coef_{i}", np.exp(cph.coef_[i]))
    # predic cph
    cph_surv = cph.predict_survival_function(pd.DataFrame(x_out))

    # get plotable data
    # cph_sv_val = [sf(np.arange(T)) for sf in cph_surv]
    cph_sv_t = cph_surv[0].x
    cph_sv_val = [sf(cph_sv_t) for sf in cph_surv]
    cph_sv_t = np.concatenate([np.array([0]), cph_sv_t])
    cph_sv_val = [np.concatenate([np.array([1]), sv]) for sv in cph_sv_val]

    # log artif plot curves
    title = "cph_surv_pred"
    ssf.plot_sv(x_mat, cph_sv_val, t=cph_sv_t, title = title, save=True, dir="outputs")
    ml.log_artifact(f"outputs/{title}.png")
    # log model cph
    # idk how to do
    
    ###################################################################################
    #  model rsf
    rsf = sksurv.ensemble.RandomSurvivalForest(
        n_estimators=1000, min_samples_split=100, min_samples_leaf=15, n_jobs=-1, random_state=20
    )
    rsf.fit(x_sk, y_sk)
    # predict rsf
    rsf_surv = rsf.predict_survival_function(pd.DataFrame(x_out))
    # get plotable predictions
    # rsf_sv_val = [sf(np.arange(T)) for sf in rsf_surv]
    rsf_sv_t = rsf_surv[0].x
    rsf_sv_val = [sf(rsf_sv_t) for sf in rsf_surv]
    rsf_sv_t = np.concatenate([np.array([0]), rsf_sv_t])
    rsf_sv_val = [np.concatenate([np.array([1]), sv]) for sv in rsf_sv_val]
    # log artif plot curves
    title = "rsf_surv_pred"
    ssf.plot_sv(x_mat, rsf_sv_val, t=rsf_sv_t, title=title, save=True, dir="outputs")
    ml.log_artifact(f"outputs/{title}.png")
    # log model resf

    ################################################################################
    # BART
    M = 200 # number of trees
    DRAWS = 200
    TUNE = 200
    CORES = 4
    ml.log_param("n_tree", M)
    ml.log_param("draws", DRAWS)
    ml.log_param("tune", TUNE)

    # tranform data long-form
    b_tr_t, b_tr_delta, b_tr_x = ssf.surv_pre_train2(x_sk, y_sk)
    # b_te_t, b_te_x = surv_pre_test(x_sk, y_sk)
    b_te_x = ssf.get_bart_test(x_out, np.unique(b_tr_t))
    off = sp.norm.ppf(np.mean(b_tr_delta))
    # model bart
  
    with pm.Model() as bart:
        x_data = pm.MutableData("x", b_tr_x)
        f = pmb.BART("f", X=x_data, Y=b_tr_delta, m=M)
        z = pm.Deterministic("z", f + off)
        mu = pm.Deterministic("mu", pm.math.invprobit(z))
        y_pred = pm.Bernoulli("y_pred", p=mu, observed=b_tr_delta, shape=x_data.shape[0])
        bdata = pm.sample(random_seed=2, draws=200, tune = 200, cores=4)

    with bart:
    # pm.set_data({"x":pd.DataFrame(test_x), "off":off_test})
        pm.set_data({"x":pd.DataFrame(b_te_x)})
        pp = pm.sample_posterior_predictive(bdata, var_names = ["y_pred", "f", "z", "mu"])
    
    # transform to survival
    bart_sv_fx = ssf.get_sv_fx(pp, x_out)
    # bart_svt
    bart_sv_t = np.unique(b_tr_t)

    # add a time 0 with prob 1 
    bart_sv_t = np.concatenate([np.array([0]), bart_sv_t])
    bart_sv_val = [np.concatenate([np.array([1]), sv]) for sv in bart_sv_fx]
    
    # log artif plot curves
    title = "bart_surv_pred"
    ssf.plot_sv(x_mat, bart_sv_val, t=bart_sv_t, title=title, save=True, dir="outputs")
    ml.log_artifact(f"outputs/{title}.png")
    # log model bart /idk how to do

    # get metrics rmse, bias
    rsf_rmse, rsf_bias, t_quant = ssf.get_metrics( f_t = rsf_sv_val, f = sv_mat[x_idx], T = rsf_sv_t[rsf_sv_t <T])

    cph_rmse, cph_bias, t_quant = ssf.get_metrics( f_t = cph_sv_val, f = sv_mat[x_idx], T = cph_sv_t[cph_sv_t < T])

    bart_rmse, bart_bias, t_quant = ssf.get_metrics(f_t = bart_sv_val, f = sv_mat[x_idx], T = bart_sv_t[bart_sv_t < T])

    # log metri cph_rmse
    # log metri cph_bias
    # log metri rsf_rmse
    # log metri rsf_bias
    # log metri bart_rmse
    # log metri bart_bias
    for i in np.arange(rsf_rmse.shape[1]):
        ml.log_metric(f"rmse_rsf_{i}", rsf_rmse[0,i])
        ml.log_metric(f"rmse_cph_{i}", cph_rmse[0,i])
        ml.log_metric(f"rmse_bart_{i}", bart_rmse[0,i])
        ml.log_metric(f"bias_rsf_{i}", rsf_bias[0,i])
        ml.log_metric(f"bias_cph_{i}", cph_bias[0,i])
        ml.log_metric(f"bias_bart_{i}", bart_bias[0,i])

    title="RMSE"
    ssf.plot_metrics(t_quant, T, rsf_rmse[0], cph_rmse[0], bart_rmse[0], title, "outputs")
    ml.log_artifact(f"outputs/{title}.png")
    title="BIAS"
    ssf.plot_metrics(t_quant, T, rsf_bias[0], cph_bias[0], bart_bias[0], "BIAS", "outputs")
    ml.log_artifact(f"outputs/{title}.png")


# End run (defaults when using with/ block)



here


Only 200 samples in chain.
Multiprocess sampling (4 chains in 4 jobs)
PGBART: [f]


Sampling 4 chains for 200 tune and 200 draw iterations (800 + 800 draws total) took 247 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
Sampling: [f, y_pred]


In [33]:
with ml.start_run(run_name = "test3"):
    mlr = ml.active_run()

In [37]:
# run_info.info
ml.end_run()

In [38]:
run_info.info

<RunInfo: artifact_uri='file:///home/twj8/pymc_bart/mlruns/950838037347002748/0ad35deb0cf949508d5bdeae000de858/artifacts', end_time=None, experiment_id='950838037347002748', lifecycle_stage='active', run_id='0ad35deb0cf949508d5bdeae000de858', run_name='test1', run_uuid='0ad35deb0cf949508d5bdeae000de858', start_time=1690925338428, status='RUNNING', user_id='twj8'>